To DO

+ ~~change landing page~~
+ ~~DONE add loading options to simulation; must explore useAsyncQueue NOPE (AsyncQueue too different from AsyncState)~~
+ ~~DONE add table; use icons for table segments; need to have column groups~~
+ ~~DONE!   need to add charts~~

+ ~~localstorage for chart values~~
+ ~~add env statement~~
+ ~~improve Chart aesthetics~~
    ~~+ window or tiles or windows with tiles~~
    ~~+ stack legend~~
    ~~+ use theme-related colors~~
    ~~+ pass appropriate x- and y-label values~~
+ ~~standardize container sizes and alignments
+ improve aesthetics of environment section
    + ~~improve chart layout~~
    + improve summary layout
+ ~~improve Table aesthetics~~
    + ~~add units for rows in Summary~~
+ ~~use Celery and Redis for async tasks to save monthly / quarterly / annual versions of statements~~
+ ~~added celery autoreload
+ ~~MAKE NOTE THAT CHARTS ARE MONTHLY!!!!~~
+ ~~check all math~~
    + ~~why is block schedule wrong?~~
        + ~~had placeholder for current block~
    + ~~number of miners???~~
    + ~~cost of miners~~
    + ~~Why do roi and istat not overlap in terms of number of periods??????~~
    + no pool fees?
+ ~~check if statement object exist, if so, get monthly and summary only; should be much faster~~
+ ~~add current BTC environment modal~~
    + ~~feed-in live environment to enviroment assumptions~~
+ ~~no scaling done for Heat rejection products????~~
+ enviro aesthetics
    + transaction fees y-axis range needs fixing
    + enviro select load ... no user feedback
+ additional metrics:
    + compare to BTC HODL strategy
    + percentage revenue from appreciation vs. fees
+ price updates for miners
+ add Bitcoin mining demo
+ hash share should be percentage of actual hash rate, not difficult
+ ProjectSim CRUD at point of sale ... need to clean up combobox interaction
+ allow manual scaling and pricing of products
+ need to add curtailment / switching on-off of miners
+ add temperature impacts / need to feed in variable chip temp
+ time unit not working as expected
+ improve infra product database
+ improve infra section and remove errors
+ push M/Q/A statement creation INTO the serializer so that finstat resample can be used; how to get task_ids ?????
+ need full statements and charts for each project
+ need better atomicity of database transactions
+ push live to google
+ testing for drillbit
+ testing for drillbit-vj
+ testing for drillbit-dj
+ updates to finstat

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'drillbit_dj.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

updating meta...done


In [7]:
import numpy as np
import pandas as pd

In [8]:
from drillbit import BitcoinEnvironmentUtility
from drillbit.__new_objects import Rig as RigManager, Product as ProductManager, \
    Cooling as CoolingManager, Project as ProjectManager, \
    OverClock, RigOperator
from drillbit.__new_units__ import AbstractBaseUnit, MagnitudeTable, Hashes, HashRate, \
    Time, Power, Energy, Efficiency, EnergyPrice


from environment.serializers import EnvironmentSerializer as EnviroSer, BlockScheduleSerializer as BSchedSer, \
    BitcoinPriceSerializer as BPSer, TransactionFeesSerializer as FeeSer, HashRateSerializer as HRSer
from environment.models import Environment, BlockSchedule, BitcoinPrice, TransactionFees

from products.models import Rig, Cooling, HeatRejection, Electrical, RejectionCurve
from products.serializers import RigSerializer, CoolingSerializer, HeatRejectionSerializer, \
    ElectricalSerializer, RejectionCurveForProductSerializer as CurveSerializer

from projects.models import RigForProject, InfraForProject, Project, Projects, \
    ProjectStatement, ProjectSimulation, ProjectStatementSummary
from projects.serializers import RigForProjectSerializer, InfraForProjectSerializer, ProjectSerializer, ProjectsSerializer, \
    ProjectScalingSerializer, ProjectCostsSerializer, ProjectStatementSerializer, \
    ProjectStatementSummarySerializer

from projects.tasks import create_statements_for_given_project, create_statement_for_given_frequency

In [394]:
import numpy as np
from tqdm import tqdm, trange
import pandas as pd
import numpy_financial as npf

import finstat as fs

from drillbit.statements.funcs import total_energy, win_percentage, \
    expected_difficulty, hashes_to_hash_rate, win_percentage

from products.models import WeatherStation, WeatherData
from products.serializers import expected_loss_from_insufficient_cooling_capacity

In [10]:
from drillbit.statements.statements import init_environment, ProjectTemplate, ROITemplate, \
    analysis

In [519]:
projs = Projects.objects.latest('created_at')

In [522]:
sims = ProjectSimulation.objects.filter(project__in=projs.projects.all())

In [527]:
sims.delete()

(4, {'projects.ProjectSimulation': 4})

In [526]:
ProjectStatement.objects.filter(sim__in=sims).delete()

(16, {'projects.ProjectStatement': 16})

In [525]:
ProjectStatementSummary.objects.filter(sim__in=sims).delete()

(4, {'projects.ProjectStatementSummary': 4})

In [11]:
btc = BitcoinEnvironmentUtility()
environment = Environment.objects.latest('created_at')
project = Project.objects.latest('created_at')

updating meta...done


In [468]:
price = btc.current_price
price = 24000
capacity = 50e6
reward = btc.current_reward
difficulty = btc.difficulty
station = WeatherStation.objects.get(location__contains='Pecos')
rig = Rig.objects.get(make='Antminer', model='S19', generation='jPro')
rejection = HeatRejection.objects.get(design_dry_bulb=95)

In [469]:
datas = []
for rejection in HeatRejection.objects.all():
    n_coolers = capacity / rejection.capacity
    cost_of_coolers = n_coolers * rejection.price
    expected_loss = expected_loss_from_insufficient_cooling_capacity(
        reward, difficulty,
        station, rig, rejection,
        n_coolers
    )
    data = {
        'design_dry_bulb': rejection.design_dry_bulb,
        'cost_of_coolers': cost_of_coolers,
        'expected_loss': -expected_loss
    }
    datas.append(data)

In [470]:
df = pd.DataFrame(datas)

In [471]:
df.loc[:, 'incremental_gain'] = df.expected_loss.diff().fillna(0)
df.loc[:, 'cumulative_gain'] = df.incremental_gain.cumsum()
df.loc[:, 'incremental_cost'] = df.cost_of_coolers.diff().fillna(0)
df.loc[:, 'cumulative_cost'] = df.incremental_cost.cumsum()
df.loc[:, 'cumulative_gain_usd'] = df.cumulative_gain * price
df.loc[:, 'payback'] = df.cumulative_cost / df.cumulative_gain_usd

In [472]:
df.to_csv('cooler_capacity.csv')

In [473]:
os.getcwd()

'/Users/spindicate/Documents/programming/drillbit/drillbit_dj'

In [475]:
df

,design_dry_bulb,cost_of_coolers,expected_loss,incremental_gain,cumulative_gain,incremental_cost,cumulative_cost,cumulative_gain_usd,payback
0,90.0,3750000.0,-59.201713,0.000000,0.000000,0.0,0.0,0.000000e+00,NaN
1,91.0,4000000.0,-55.945689,3.256024,3.256024,250000.0,250000.0,7.814457e+04,3.199199
2,92.0,4100000.0,-48.173348,7.772341,11.028365,100000.0,350000.0,2.646808e+05,1.322348
3,93.0,4200000.0,-41.493829,6.679519,17.707884,100000.0,450000.0,4.249892e+05,1.058850
4,94.0,4250000.0,-38.456240,3.037588,20.745472,50000.0,500000.0,4.978913e+05,1.004235
5,95.0,4425000.0,-32.808636,5.647604,26.393077,175000.0,675000.0,6.334338e+05,1.065620
6,96.0,4700000.0,-29.283495,3.525141,29.918218,275000.0,950000.0,7.180372e+05,1.323051
7,97.0,4825000.0,-25.400033,3.883463,33.801680,125000.0,1075000.0,8.112403e+05,1.325131
8,98.0,5050000.0,-21.063444,4.336588,38.138269,225000.0,1300000.0,9.153184e+05,1.420271
9,99.0,5575000.0,-17.145141,3.918303,42.056572,525000.0,1825000.0,1.009358e+06,1.808080
